In [13]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from darksky import forecast
from datetime import datetime as dt

from modules.nyc_shapefiles import read_nyc_shapefiles

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
nypd_data = load_clean_felonies()

/Users/jsamet/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
precinct_dict, tract_dict, merged_census_info = read_nyc_shapefiles()

In [3]:
def get_precinct_centroids(precinct_dict):
    return {
        k: {
            'latitude': v['shape_gps'].centroid.y,
            'longitude': v['shape_gps'].centroid.x
        }
        for k, v in precinct_dict.items()
    }

In [4]:
def get_mean_latlon(centroids):
    arr = [[v['latitude'], v['longitude']] for v in centroids.values()]
    return np.mean(arr, axis=0)

In [5]:
centroids = get_precinct_centroids(precinct_dict)
nyc_mean_latlon = get_mean_latlon(centroids)

In [6]:
with open('../precrime_data/darksky_api_key') as f:
    darksky_api_key = f.read().replace('\n','')
    
t = dt(2017, 5, 13, 14).isoformat()
NYC = darksky_api_key, nyc_mean_latlon[0], nyc_mean_latlon[1]
with forecast(*NYC, time=t) as nyc_weather:
    for k, v in nyc_weather.currently._data.items():
        print('{0}:'.format(k), v)


time: 1494698400
summary: Rain
icon: rain
precipIntensity: 0.0945
precipProbability: 0.92
precipType: rain
temperature: 50.89
apparentTemperature: 50.89
dewPoint: 48.43
humidity: 0.91
pressure: 1010.78
windSpeed: 13.42
windBearing: 72
cloudCover: 1
uvIndex: 5
visibility: 2.97


In [7]:
def load_clean_felonies(data_path=None, clean_file=None):
    clean_file_defaults = {
        'data_path': '../precrime_data/',
        'clean_file': 'clean_felonies.csv',
        'dtype': {
            'CMPLNT_NUM': np.int64,
            'COMPLAINT_DATETIME': str,
            'REPORT_DATE': str,
            'KY_CD': np.int32,
            'OFNS_DESC': str,
            'BORO_NM': str,
            'ADDR_PCT_CD': np.int32,
            'Latitude': np.float64,
            'Longitude': np.float64,
        },
        'index_col': 'CMPLNT_NUM',
        'usecols': [
            'CMPLNT_NUM',
            'COMPLAINT_DATETIME',
            'REPORT_DATE',
            'KY_CD',
            'OFNS_DESC',
            'BORO_NM',
            'ADDR_PCT_CD',
            'Latitude',
            'Longitude',
        ],
        'parse_dates_cols': ['REPORT_DATE', 'COMPLAINT_DATETIME'],
    }
    
    if data_path is None:
        data_path = clean_file_defaults['data_path']
    if clean_file is None:
        clean_file = clean_file_defaults['clean_file']
        
    nypd_data = pd.read_csv(
        filepath_or_buffer = data_path+clean_file,
        index_col = clean_file_defaults['index_col'],
        usecols = clean_file_defaults['usecols'],
        dtype = clean_file_defaults['dtype'],
        parse_dates = clean_file_defaults['parse_dates_cols'],
        infer_datetime_format = True,
    )
    
    nypd_data.sort_values(by='COMPLAINT_DATETIME', inplace=True)
    return nypd_data

In [9]:
nypd_dates = nypd_data['COMPLAINT_DATETIME'].dt.date.unique()

In [17]:
hours = range(2,26,4)
write_header = True
with open('../precrime_data/weather_hist.csv', 'a', newline='') as csvfile:
    with forecast(*NYC) as current_weather:
        weather_fields = list(set(
            ['precipType'] + 
            list(current_weather.currently._data.keys())
        ))
    writer = csv.DictWriter(csvfile, fieldnames=weather_fields)
    if write_header:
        writer.writeheader()
    for date in nypd_dates:
        for hour in hours:
            t = dt(date.year, date.month, date.day, hour).isoformat()
            with forecast(*NYC, time=t) as nyc_weather:
                writer.writerow({k: v for k, v in nyc_weather.currently._data.items()})


In [11]:
for hour in hours:
    print(hour)

2
6
10
14
18
22


In [18]:
import pandas as pd

In [42]:
hist_w = pd.read_csv('../precrime_data/weather_hist.csv', index_col='time')
hist_w['nyctime'] = pd.to_datetime(hist_w.index, unit='s').tz_localize('UTC').tz_convert('America/New_York')
hist_w.set_index('nyctime', inplace=True)

hist_w

,windBearing,icon,visibility,precipProbability,temperature,windGust,summary,nearestStormBearing,precipType,ozone,cloudCover,windSpeed,uvIndex,dewPoint,humidity,apparentTemperature,precipIntensity,pressure,nearestStormDistance
nyctime,,,,,,,,,,,,,,,,,,,
2006-01-02 02:00:00-05:00,243,clear-night,9.35,0.00,37.62,NaN,Clear,NaN,NaN,NaN,0.24,3.96,0.0,31.26,0.78,34.61,0.0000,1023.60,NaN
2006-01-02 06:00:00-05:00,220,partly-cloudy-night,9.52,0.00,38.55,NaN,Mostly Cloudy,NaN,NaN,NaN,0.84,4.35,0.0,32.01,0.77,35.32,0.0000,1023.18,NaN
2006-01-02 10:00:00-05:00,184,partly-cloudy-day,9.38,0.00,41.55,NaN,Partly Cloudy,NaN,NaN,NaN,0.43,3.82,1.0,33.17,0.72,39.27,0.0000,1024.20,NaN
2006-01-02 14:00:00-05:00,153,partly-cloudy-day,9.74,0.00,44.41,NaN,Mostly Cloudy,NaN,NaN,NaN,0.63,3.40,1.0,31.98,0.61,42.95,0.0000,1021.33,NaN
2006-01-02 18:00:00-05:00,96,rain,4.60,1.00,40.52,NaN,Rain,NaN,rain,NaN,1.00,8.29,0.0,36.84,0.87,35.07,0.0737,1020.90,NaN
2006-01-02 22:00:00-05:00,67,rain,5.30,1.00,39.62,NaN,Light Rain,NaN,rain,NaN,1.00,11.65,0.0,37.54,0.92,32.50,0.0512,1016.11,NaN
2006-01-03 02:00:00-05:00,64,rain,3.60,1.00,40.33,NaN,Rain and Breezy,NaN,rain,NaN,1.00,19.50,0.0,38.17,0.92,31.03,0.1284,1011.36,NaN
2006-01-03 06:00:00-05:00,47,rain,4.38,1.00,37.25,NaN,Light Rain,NaN,rain,NaN,1.00,17.06,0.0,35.08,0.92,27.68,0.0509,1010.34,NaN
2006-01-03 10:00:00-05:00,46,rain,2.93,0.96,36.68,NaN,Light Rain,NaN,rain,NaN,1.00,16.37,1.0,34.24,0.91,27.15,0.0297,1011.98,NaN


In [41]:
hist_w

,windBearing,icon,visibility,precipProbability,temperature,windGust,summary,nearestStormBearing,precipType,ozone,cloudCover,windSpeed,uvIndex,dewPoint,humidity,apparentTemperature,precipIntensity,pressure,nearestStormDistance
humantime,,,,,,,,,,,,,,,,,,,
2006-01-02 02:00:00-05:00,243,clear-night,9.35,0.00,37.62,NaN,Clear,NaN,NaN,NaN,0.24,3.96,0.0,31.26,0.78,34.61,0.0000,1023.60,NaN
2006-01-02 06:00:00-05:00,220,partly-cloudy-night,9.52,0.00,38.55,NaN,Mostly Cloudy,NaN,NaN,NaN,0.84,4.35,0.0,32.01,0.77,35.32,0.0000,1023.18,NaN
2006-01-02 10:00:00-05:00,184,partly-cloudy-day,9.38,0.00,41.55,NaN,Partly Cloudy,NaN,NaN,NaN,0.43,3.82,1.0,33.17,0.72,39.27,0.0000,1024.20,NaN
2006-01-02 14:00:00-05:00,153,partly-cloudy-day,9.74,0.00,44.41,NaN,Mostly Cloudy,NaN,NaN,NaN,0.63,3.40,1.0,31.98,0.61,42.95,0.0000,1021.33,NaN
2006-01-02 18:00:00-05:00,96,rain,4.60,1.00,40.52,NaN,Rain,NaN,rain,NaN,1.00,8.29,0.0,36.84,0.87,35.07,0.0737,1020.90,NaN
2006-01-02 22:00:00-05:00,67,rain,5.30,1.00,39.62,NaN,Light Rain,NaN,rain,NaN,1.00,11.65,0.0,37.54,0.92,32.50,0.0512,1016.11,NaN
2006-01-03 02:00:00-05:00,64,rain,3.60,1.00,40.33,NaN,Rain and Breezy,NaN,rain,NaN,1.00,19.50,0.0,38.17,0.92,31.03,0.1284,1011.36,NaN
2006-01-03 06:00:00-05:00,47,rain,4.38,1.00,37.25,NaN,Light Rain,NaN,rain,NaN,1.00,17.06,0.0,35.08,0.92,27.68,0.0509,1010.34,NaN
2006-01-03 10:00:00-05:00,46,rain,2.93,0.96,36.68,NaN,Light Rain,NaN,rain,NaN,1.00,16.37,1.0,34.24,0.91,27.15,0.0297,1011.98,NaN


In [43]:
with forecast(*NYC) as current_weather:
    for k, v in current_weather.currently._data.items():
        print('{0}: '.format(k), v)


time:  1509981230
summary:  Overcast
icon:  cloudy
nearestStormDistance:  8
nearestStormBearing:  2
precipIntensity:  0
precipProbability:  0
temperature:  62.94
apparentTemperature:  63.2
dewPoint:  60.19
humidity:  0.91
pressure:  1013.27
windSpeed:  5.18
windGust:  8.34
windBearing:  239
cloudCover:  0.99
uvIndex:  1
visibility:  8.6
ozone:  250.42
